# **Simple CNN**

## **Imports**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder

from utils import load_images
from utils import TrainingPlot

## **Parameters**

In [ ]:
MODEL_VERSION = "_cnn_1.1"

METADATA = pd.read_csv('../src/meta_data.csv')
IMAGE_FOLDER = "../src/final_data"
IMG_HEIGHT = 128
IMG_WIDTH = 128

## **Load Images**

In [ ]:
METADATA = pd.read_csv('../src/meta_data.csv')

# Load images and labels
image_folder = '../src/final_data'
images, labels = load_images(image_folder, METADATA, IMG_HEIGHT, IMG_WIDTH)

# Normalize images
images = images / 255.0

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

# Train / Test Split
X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)

## **Create Model**

In [ ]:
def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)
num_classes = len(label_encoder.classes_)
model = create_cnn_model(input_shape, num_classes)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## **Training**

In [ ]:
# Train the model
history = model.fit(X_train, y_train, 
                    epochs=12, 
                    validation_split=0.3, 
                    batch_size=32,
                    callbacks=[TrainingPlot()])

In [ ]:
model.save(f"../src/models/model_v{MODEL_VERSION}.h5")
model.save(f"../src/models/model_v{MODEL_VERSION}.keras")

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')

In [ ]:
del X_train
del X_test
del y_train
del y_test
del images
del labels
del label_encoder
del labels_encoded
del model